# **Türkçe Belgeler Üzerinde Anlamsal Soru-Cevaplama Sistemi: Geliştirme Kodları - 1**

Aşağıda, farklı başlıklar altında yer alan kodlar ve açıklamalar, sistemin nasıl çalıştığını adım adım göstermek ve geliştirme sürecini detaylandırmak amacıyla sunulmuştur. Ayrıca **4 numaralı versiyon mevcut notebookda ilerlemenin güncel ve son halidir**.

# **1- BM25 (Sparse Retriever)**
Bu kodda, PDF’ten metni çıkarıp küçük parçalara bölerek bellekte tutan (InMemoryDocumentStore) ve
bu metin parçalarını BM25 algoritmasıyla hızlıca arayan bir retriever ile, bulunan en alakalı parçadan
soruya doğrudan metinden cevap çıkaran (extractive QA) bir FARMReader kullanılıyor. Yani burada klasik
“soruya en uygun metin parçasını bul ve içinden cevabı kes” yaklaşımı var. RAG (Retrieval-Augmented Generation)
ise sadece metinden alıntı yapmaz, ayrıca bulduğu bilgiyi doğal dilde yeni cümleler üreterek cevap oluşturur; bu nedenle
daha karmaşık bir generatif model ve genellikle farklı pipeline yapısı gerektirir. Ayrıca RAG, genelde GPU ve güçlü donanım ister,
Türkçe için iyi eğitilmiş RAG modelleri yaygın olmadığından, bu projede basit, hızlı ve yerel çalışabilen extractive QA yaklaşımı tercih edilmiştir.

In [ ]:
pip install farm-haystack pdfplumber pymupdf

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import BM25Retriever, FARMReader
from haystack.schema import Document
import pdfplumber
import fitz

# PDF’ten metin çıkar

def extract_text_from_pdf(pdf_path):
    full_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                # sayfa metninde gereksiz boşluklar varsa temizle
                page_text = page_text.replace('\xa0', ' ')  # görünmeyen boşlukları düzelt
                full_text += page_text + "\n"
    # print(full_text)
    return full_text

"""
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        text = page.get_text()
        full_text += text + "\n"
    return full_text
"""
# Belgeyi parçalara ayır
def prepare_documents(text, chunk_size=150):
    # fazla boşlukları tek boşluğa indir
    import re
    clean_text = re.sub(r'\s+', ' ', text).strip()
    words = clean_text.split(' ')
    return [Document(content=" ".join(words[i:i + chunk_size])) for i in range(0, len(words), chunk_size)]


# Yükle
text = extract_text_from_pdf("aa.pdf")
documents = prepare_documents(text)

# Haystack yapısı (FAISS yerine InMemory + BM25 kullan)
document_store = InMemoryDocumentStore(use_bm25=True)
document_store.write_documents(documents)

retriever = BM25Retriever(document_store=document_store)
reader = FARMReader(model_name_or_path="savasy/bert-base-turkish-squad", use_gpu=False)

pipe = ExtractiveQAPipeline(reader, retriever)

soru = "Raporun sonucu nedir?"
result = pipe.run(query=soru, params={"Retriever": {"top_k": 3}, "Reader": {"top_k": 1}})
print("\n\nCevap:", result["answers"][0].answer)


# **2- Dense Retriever (FAISS + Embedding)**
Bu versiyon, Türkçe PDF belgeleri üzerinde anlamsal bilgi erişimi ve soru-cevaplama işlemleri gerçekleştirmek için Haystack kütüphanesini kullanır.
PDF'ten çıkarılan metinler, anlam bütünlüğü korunacak şekilde parçalara ayrılır ve FAISS tabanlı bir bellek içi vektör veritabanına kaydedilir. Ardından, emrecan/bert-base-turkish-cased-mean-nli-stsb-tr modeliyle her belge embedding'e dönüştürülür ve semantic search yapılır. Kullanıcının sorduğu soruya en alakalı segmentler seçildikten sonra, savasy/bert-base-turkish-squad modeliyle bu segmentler üzerinde doğrudan cevap tahmini yapılır.Bu yapı, kelime eşleşmesine dayalı klasik yöntemlerin aksine, metnin anlamını dikkate alarak daha isabetli sonuçlar üretir.


In [ ]:
!apt-get update
!apt-get install -y libfaiss-dev libfaiss-dev libomp-dev

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install sqlalchemy==1.3.24

In [ ]:
!pip install farm-haystack pypdf

In [ ]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.schema import Document
import re
from pypdf import PdfReader

# PDF'ten metin çıkaran fonksiyon
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    full_text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            page_text = page_text.replace('\xa0', ' ')
            full_text += page_text + "\n"
    return full_text

# Metni küçük parçalara bölen fonksiyon
def prepare_documents(text, chunk_size=150):
    clean_text = re.sub(r'\s+', ' ', text).strip()
    words = clean_text.split(' ')
    return [Document(content=" ".join(words[i:i+chunk_size])) for i in range(0, len(words), chunk_size)]

# PDF dosyasından metin al
text = extract_text_from_pdf("aa.pdf"),
documents = prepare_documents(text)

# FAISS Document Store RAM üzerinde oluştur (sql_url = "sqlite://" demek RAM'de demek)
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", sql_url="sqlite://")

# Retriever oluştur (embedding modeli, GPU kullanma)
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="emrecan/bert-base-turkish-cased-mean-nli-stsb-tr", # sentence-transformers/paraphrase-xlm-r-multilingual-v1
    use_gpu=True
)

# Dokümanları veritabanına yaz
document_store.write_documents(documents)

# Dokümanlar için embeddingleri hesapla ve indexi güncelle
document_store.update_embeddings(retriever)

# Reader modeli (Türkçe SQuAD)
reader = FARMReader(model_name_or_path="savasy/bert-base-turkish-squad", use_gpu=True)

# Soru-Cevap pipeline'ını oluştur
pipe = ExtractiveQAPipeline(reader, retriever)

# Soru sor
soru = "raporun sonucu nedir?"
result = pipe.run(query=soru, params={"Retriever": {"top_k": 3}, "Reader": {"top_k": 1}})

print("\n\nCevap:", result["answers"][0].answer)


# **3- Dense Retriever (FAISS + Embedding)**
**Daha gelişmiş hali mevcut hem test açısıdan hem kod yapısından (temizleme işlemleri arttırıldı)**

In [ ]:
!apt-get update
!apt-get install -y libfaiss-dev libfaiss-dev libomp-dev
!pip install faiss-cpu
!pip install sqlalchemy==1.3.24
!pip install farm-haystack pypdf pymupdf

In [ ]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.schema import Document
import re
from pypdf import PdfReader
import pymupdf  # PyMuPDF alternatif olarak
import unicodedata

def extract_text_from_pdf_improved(pdf_path):
    """
    Geliştirilmiş PDF metin çıkarma fonksiyonu
    Önce PyMuPDF dener, sonra pypdf'e geçer
    """
    try:
        # PyMuPDF ile deneme (genellikle daha iyi Türkçe desteği)
        doc = pymupdf.open(pdf_path)
        full_text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text = page.get_text()
            if text:
                # Unicode normalizasyonu
                text = unicodedata.normalize('NFKD', text)
                full_text += text + "\n"
        doc.close()

        if full_text.strip():
            return full_text
    except Exception as e:
        print(f"PyMuPDF hatası: {e}, pypdf'e geçiliyor...")

    # pypdf ile deneme
    try:
        reader = PdfReader(pdf_path)
        full_text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                # Karakter temizleme
                page_text = page_text.replace('\xa0', ' ')
                page_text = page_text.replace('\u00a0', ' ')
                # Unicode normalizasyonu
                page_text = unicodedata.normalize('NFKD', page_text)
                full_text += page_text + "\n"
        return full_text
    except Exception as e:
        print(f"pypdf hatası: {e}")
        return ""

def clean_text(text):
    """Metni temizleme ve normalize etme"""
    # Gereksiz boşlukları temizle
    text = re.sub(r'\s+', ' ', text)

    # Satır sonlarını düzelt
    text = re.sub(r'\n\s*\n', '\n\n', text)

    # Bozuk karakterleri düzelt (örnekler)
    replacements = {
        '¸c': 'ç',
        '¸s': 'ş',
        '˘g': 'ğ',
        '¨u': 'ü',
        '¨o': 'ö',
        '˙I': 'İ',
        'ı¸': 'ış',
        'C¸': 'Ç',
        'S¸': 'Ş',
        'G˘': 'Ğ',
        'U¨': 'Ü',
        'O¨': 'Ö'
    }

    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)

    return text.strip()

def prepare_documents_improved(text, chunk_size=200, overlap=50):
    """
    Geliştirilmiş doküman hazırlama - overlap ile
    """
    clean_text_content = clean_text(text)

    # Cümle bazlı bölme (daha mantıklı)
    sentences = re.split(r'[.!?]+', clean_text_content)
    sentences = [s.strip() for s in sentences if s.strip()]

    documents = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) < chunk_size:
            current_chunk += sentence + ". "
        else:
            if current_chunk:
                documents.append(Document(content=current_chunk.strip()))

            # Overlap için son cümleyi sakla
            if overlap > 0:
                current_chunk = sentence + ". "
            else:
                current_chunk = ""

    # Son chunk'ı ekle
    if current_chunk:
        documents.append(Document(content=current_chunk.strip()))

    return documents

def setup_rag_pipeline(pdf_path):
    """RAG pipeline'ını kuran ana fonksiyon"""

    print("PDF'den metin çıkarılıyor...")
    text = extract_text_from_pdf_improved(pdf_path)

    if not text.strip():
        raise ValueError("PDF'den metin çıkarılamadı!")

    print(f"Çıkarılan metin uzunluğu: {len(text)} karakter")

    # İlk 500 karakteri kontrol için yazdır
    print("\nÇıkarılan metnin ilk 500 karakteri:")
    print(text[:500])
    print("\n" + "="*50)

    # Dokümanları hazırla
    documents = prepare_documents_improved(text)
    print(f"Toplam {len(documents)} doküman parçası oluşturuldu")

    # FAISS Document Store
    document_store = FAISSDocumentStore(
        faiss_index_factory_str="Flat",
        sql_url="sqlite://"
    )

    # Retriever (Türkçe için optimize edilmiş model)
    retriever = EmbeddingRetriever(
        document_store=document_store,
        embedding_model="emrecan/bert-base-turkish-cased-mean-nli-stsb-tr",
        use_gpu=True
    )

    # Dokümanları yaz
    print("Dokümanlar veritabanına yazılıyor...")
    document_store.write_documents(documents)

    # Embeddings hesapla
    print("Embeddings hesaplanıyor...")
    document_store.update_embeddings(retriever)

    # Reader
    reader = FARMReader(
        model_name_or_path="savasy/bert-base-turkish-squad",
        use_gpu=True
    )

    # Pipeline
    pipeline = ExtractiveQAPipeline(reader, retriever)

    print("RAG sistemi hazır!")
    return pipeline

def ask_question(pipeline, question, top_k_retriever=5, top_k_reader=3):
    """Soru sorma fonksiyonu"""
    print(f"\nSoru: {question}")
    print("-" * 40)

    result = pipeline.run(
        query=question,
        params={
            "Retriever": {"top_k": top_k_retriever},
            "Reader": {"top_k": top_k_reader}
        }
    )

    if result["answers"]:
        for i, answer in enumerate(result["answers"]):
            print(f"\nCevap {i+1}: {answer.answer}")
            print(f"Güven skoru: {answer.score:.3f}")
            print(f"Kaynak: {answer.context[:200]}...")
    else:
        print("Cevap bulunamadı.")

    return result

# Ana kullanım
if __name__ == "__main__":
    try:
        # Pipeline'ı kur
        pipeline = setup_rag_pipeline("aa.pdf")

        # Örnek sorular
        questions = [
            "Raporun sonucu nedir?",
            "MinerU neden tercih edilmiş?",
            "Hangi araçlar karşılaştırılmış?",
            "OCR desteği olan araçlar hangileri?",
            "Türkçe desteği nasıl?"
        ]

        for question in questions:
            ask_question(pipeline, question)
            print("\n" + "="*60 + "\n")

    except Exception as e:
        print(f"Hata: {e}")
        print("PyMuPDF kurulu değilse: pip install PyMuPDF")

# **4- Dense Retriever (FAISS + Embedding)**
**Bu versiyon da, pdf okumada Türkçe desteği sağlamak için haystack içinden pdf okuma işlemi yapılmıştır. Ek olarak prepare_documents fonksiyonu daha gelişmiş hale getirilmiştir.**

In [ ]:
!apt-get update
!apt-get install -y libfaiss-dev libfaiss-dev libomp-dev
!pip install faiss-cpu
!pip install sqlalchemy==1.3.24
!pip install farm-haystack pypdf pymupdf

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,801 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,063 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,040 kB]
Get:13 http://archive.ubuntu.com/ubunt

In [ ]:
!apt-get install -y poppler-utils
!pip uninstall langdetect -y
!pip uninstall farm-haystack -y
!pip install farm-haystack[preprocessing]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 36 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 1s (183 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126492 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...
Found existing installation: farm-haystack 1.26.4.post0
Uninstalling farm-haystack-1.26.4.post0:
  Successfully uninstalled farm-haystack-1.26.4.post0
  Using cached farm_haystack-1.26.4.po

In [ ]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.schema import Document
from haystack.nodes import PDFToTextConverter
import re

# PDF'den metin çıkarma için Haystack converter kullan
def extract_text_with_haystack_converter(pdf_path):
    converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["tr"])  # Türkçe desteği için
    documents = converter.convert(file_path=pdf_path, meta=None)

    # documents liste olarak geliyor, her biri Document objesi
    full_text = " ".join([doc.content for doc in documents])
    return full_text

"""
def prepare_documents(text, chunk_size=150):
    clean_text = re.sub(r'\s+', ' ', text).strip()
    words = clean_text.split(' ')
    return [Document(content=" ".join(words[i:i+chunk_size])) for i in range(0, len(words), chunk_size)]

"""
def prepare_documents(text, chunk_size=500, overlap=100):
    # Önce kelimelere böl
    words = re.sub(r'\s+', ' ', text).strip().split()
    chunks = []

    for i in range(0, len(words), chunk_size - overlap):
        chunk_words = words[i:i + chunk_size]
        chunk_text = " ".join(chunk_words)

        # Chunk'ın sonunu cümle sınırına yakınlaştır
        if i + chunk_size < len(words):  # Son chunk değilse
            # Son 10 kelime içinde nokta ara
            last_part = " ".join(chunk_words[-10:])
            if '.' in last_part:
                sentence_end = chunk_text.rfind('.')
                if sentence_end > len(chunk_text) * 0.8:  # %80'den sonra nokta varsa
                    chunk_text = chunk_text[:sentence_end + 1]

        chunks.append(Document(content=chunk_text))

    return chunks

# PDF'den metin çıkar
text = extract_text_with_haystack_converter("Arıza Analiz Yöntemleri Ve Arıza Giderme.pdf")
documents = prepare_documents(text)

# DocumentStore oluştur
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", sql_url="sqlite://")

# Retriever ayarla
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="emrecan/bert-base-turkish-cased-mean-nli-stsb-tr",
    use_gpu=True
)

# Dokümanları yaz ve embeddingleri güncelle
document_store.write_documents(documents)
document_store.update_embeddings(retriever)

# Reader
reader = FARMReader(model_name_or_path="savasy/bert-base-turkish-squad", use_gpu=True, return_no_answer=False)

# Pipeline oluştur
pipe = ExtractiveQAPipeline(reader, retriever)

# Soru sor
soru = "Kondansatör kısa devre olduğunda nasıl bir belirti verir?"
result = pipe.run(query=soru, params={"Retriever": {"top_k": 3}, "Reader": {"top_k": 3}})

for ans in result["answers"]:
    print("\nCevap:", ans.answer)
    print("Confidence:", ans.score)


Writing Documents: 10000it [00:00, 221234.90it/s]        


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Updating Embedding:   0%|          | 0/28 [00:00<?, ? docs/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Documents Processed: 10000 docs [00:00, 12215.71 docs/s]
Some weights of the model checkpoint at savasy/bert-base-turkish-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.71 Batches/s]


Cevap: Kısa devre olmuş bir diyot her iki yönde de çok küçük direnç gösterecek demektir.
Confidence: 0.9932705760002136

Cevap: çıkış 0 (sıfır) volt olur
Confidence: 0.9558330178260803

Cevap: Şekilde bir direnç kısa devre olmuştur. Voltmetrede okunan değere göre arızalı elemanı bulunuz.
Confidence: 0.8817804455757141


# MinerU Kurulumu
Daha gelişmiş bir pdf okuma sistemi olarak denemeler yapmak ve tablo yapılarını çıkarmak için kurulum gerçekleştirildi ama şuan çok iyi entegre edilemiyor bundan dolayı alternatif farklı daha az maliyetli tool'lar araştırılıp tablo yapısını çıkarmak için testler yapılmalıdır.

In [ ]:
pip install huggingface_hub

In [ ]:
!pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 115.4 MB/s eta 0:00:00


In [ ]:
!uv pip install -U "mineru[all]"

Görüntülenen çıkış son 5000 satıra kısaltıldı.
botocore                 ------------------------------ 9.89 MiB/13.08 MiB
llguidance               ------------------------------ 10.35 MiB/14.31 MiB
onnxruntime              ------------------------------ 10.47 MiB/15.63 MiB
numpy                    ------------------------------ 10.33 MiB/16.15 MiB
nvidia-nvjitlink-cu12    ------------------------------ 10.26 MiB/18.83 MiB
nvidia-cuda-nvrtc-cu12   ------------------------------ 10.54 MiB/22.55 MiB
scipy                    ------------------------------ 10.30 MiB/33.65 MiB
pyarrow                  ------------------------------ 10.40 MiB/40.34 MiB
nvidia-curand-cu12       ------------------------------ 10.31 MiB/53.67 MiB
triton                   ------------------------------ 10.33 MiB/148.48 MiB
nvidia-cusparselt-cu12   ------------------------------ 10.41 MiB/149.52 MiB
nvidia-cusolver-cu12     ------------------------------ 10.21 MiB/150.90 MiB
nvidia-cufft-cu12        --------------

In [ ]:
!mineru-models-download

Please select the model download source:  (huggingface, modelscope) [huggingface]: huggingface
Please select the model type to download:  (pipeline, vlm, all) [all]: all
Fetching 1 files:   0% 0/1 [00:00<?, ?it/s]
(…)ut_yolo_docstructbench_imgsz1280_2501.pt:   0% 0.00/39.8M [00:00<?, ?B/s]
(…)ut_yolo_docstructbench_imgsz1280_2501.pt: 100% 39.8M/39.8M [00:00<00:00, 248MB/s]
Fetching 1 files: 100% 1/1 [00:00<00:00,  2.35it/s]
Fetching 1 files:   0% 0/1 [00:00<?, ?it/s]
yolo_v8_ft.pt:   0% 0.00/350M [00:00<?, ?B/s]
yolo_v8_ft.pt:   6% 21.0M/350M [00:00<00:01, 170MB/s]
yolo_v8_ft.pt:  15% 52.4M/350M [00:00<00:01, 214MB/s]
yolo_v8_ft.pt:  27% 94.4M/350M [00:00<00:00, 277MB/s]
yolo_v8_ft.pt:  39% 136M/350M [00:00<00:00, 314MB/s] 
yolo_v8_ft.pt:  51% 178M/350M [00:00<00:00, 343MB/s]
yolo_v8_ft.pt:  63% 220M/350M [00:00<00:00, 344MB/s]
yolo_v8_ft.pt:  75% 262M/350M [00:00<00:00, 352MB/s]
yolo_v8_ft.pt:  87% 304M/350M [00:00<00:00, 364MB/s]
yolo_v8_ft.pt: 100% 350M/350M [00:01<00:00, 336MB/s]
F

In [ ]:
!pip uninstall -y torchvision
!pip install torch==2.5.0
!pip install torchvision

Found existing installation: torchvision 0.22.1
Uninstalling torchvision-0.22.1:
  Successfully uninstalled torchvision-0.22.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 141.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.6 MB/s eta 0

In [ ]:
!mineru -p "aa.pdf" -o "out" --source local --method ocr --lang ch    # normalde latin olması lazım langın

2025-06-26 12:47:48.344710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750942068.364617   13486 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750942068.370683   13486 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-26 12:47:48.390601: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attrib

# Testler
Bu bölümde gerçekleştirilen testler doğrultusunda, yukarıda belirtilen en güncel versiyon üzerinde gerekli düzenlemeler yapılacaktır.

In [ ]:
sorular = [

          "Arızalı bir kondansatörün kısa devre olduğunu nasıl anlayabiliriz?",
          "Arıza gidermede sinyal izleme metodu nasıl uygulanır?",
          "Transformatörün sağlam olup olmadığını nasıl kontrol ederiz?",
          "Bir lambanın yanmama nedeni nasıl tespit edilir?",
          "Kondansatör neden açık devre olabilir?"
           ]
sayi = 1
for soru in sorular:
  result = pipe.run(query=soru, params={"Retriever": {"top_k": 3}, "Reader": {"top_k": 3}})
  print(f"\nTest {sayi}")
  print("\nSoru: "+ soru)
  for ans in result["answers"]:
      print("\nCevap:", ans.answer)
      print("Confidence:", ans.score)
  print("\n\n\n")
  sayi += 1

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.45 Batches/s]


Test 1

Soru: Arızalı bir kondansatörün kısa devre olduğunu nasıl anlayabiliriz?

Cevap: Kısa devre olmuş bir diyot her iki yönde de çok küçük direnç gösterecek demektir.
Confidence: 0.999724268913269

Cevap: Kısa devre filtre kondansatörü: Filtre kondansatörü kısa devre olmuşsa çıkış 0 (sıfır) volt olur.
Confidence: 0.9651265144348145

Cevap: Filtre girişinin şase ile kısa devre olmadığını kabul edelim. O zaman arıza diyot devresine inmiş oldu.
Confidence: 0.9580875635147095






Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.80 Batches/s]

...

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.65 Batches/s]


Test 4

Soru: Bir lambanın yanmama nedeni nasıl tespit edilir?

Cevap: Her bir direncin değerinin ölçülmesi ile veya her direncin üzerine düşen gerilimin ölçülmesiyle
Confidence: 0.9909075498580933

Cevap: Arızayı tespit ediniz.
Confidence: 0.6899492740631104

Cevap: Bu durumda ilk önce yanmayan lamba kontrol edilir. Daha sonra lambaya gelen kablolar kontrol edilir. Gerekirse hat yenilenir.
Confidence: 0.619067907333374






Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.75 Batches/s]


Test 5

Soru: Kondansatör neden açık devre olabilir?

Cevap: aşırı akım sigortasının atmasına sebep olmuşsa çıkış gerilimi sıfırdır.
Confidence: 0.9635211825370789

Cevap: Eğer R1 ve TV setinin yük direnci üzerine gerilim gelmiyorsa giriş gerilimi açık devre olan R2 direnci üzerinde görülecektir. Bu yüzden bu seri devre Kirşof’un (Kirchhoff) Gerilim Kanunu’na uyar:  Seri devrede eleman değerinin zamanla değişmesi Dirençlerin aşırı akımdan dolayı şiddetli bir baskıya maruz kalmadıkça
Confidence: 0.7992563247680664

Cevap: Eğer ani artış direnci yoksa veya doğrultmaç devresinde uygun bir sigorta kullanılmamışsa
Confidence: 0.7867424488067627






# Sonuç
*Buradaki sonuç yeni testler ve teknikler kullanıldıkça güncellenecektir.*

Mevcut son seçilen teknik şuan bazı sorulara cevap verebilirken bazı sorularda yetersiz kalmaktadır ve kısa cevaplar üretmektedir. Bazı sorularda (madde içeren sorular) yalnızca bir madde getirmektedir. Bazı sorulara tamamen yanlış cevapta getirmektedir. Mevcut pdf okuma da tabloları şuan çok iyi okuyamadığı için soruların cevabı tablolarda ise düzgün cevap gelmemektedir.

Burdaki sonuca göre yapılacak geliştirmelerden biri mevcut modellerin fine-tune edilerek daha iyi hale getirilmesi sağlanabilir.

Son olarak burada alınan sonuçların yetersizliğinden farklı bir mimari üzerinde geliştirme kodlarına geçilmiştir
